In [13]:
import pandas as pd
import numpy as np
import pickle
from multiprocessing import cpu_count
import concurrent.futures
from likelihoods.parth_exact.pyrthenope import Pyrthenope

Params

In [14]:
# Parameters

job_name = f"BBN_paper3"
save_file = f'data_files/{job_name}.pkl'

save = True

parth_inout_map = {
    'ETA10': 'eta10',
    'DNNU': 'N_nu',
    'TAU': 'tau'
}

# !!! will need to update card_mod if you change obs
# k, v of intrested obs: expermental errors 
observables = {
    'H2/H': 0.03* 10**(-5),     # https://arxiv.org/abs/1710.11129  / https://arxiv.org/pdf/1801.08023.pdf     
    #'Y_p': 0.0026,     # https://arxiv.org/abs/1904.01594 8 26 22  Izotov error 
    'Y_p': 0.0034, #0.004, # https://arxiv.org/pdf/1801.08023.pdf Aver error
    # 'tau': 30          #pdg        
}

observables = {
    'H2/H': 0.00000030,      
    'Y_p': 0.0040, 
}

# params to vary over and step size, uses parth's card syntax
parameters = {
    'ETA10': 0.0012,
    'DNNU':  0.001, 
    'TAU':   0.25
}

# cleans up output
obs_and_params = list(observables.keys()) + [parth_inout_map.get(k, k) for k in parameters.keys()]

# these are to match lesnpower output so we can just use the cobaya dali program
lo_spec_id = 'unlensed'
lo_experment = 0 

Running Function

In [15]:
def _run(param, mod=None):
    nope = Pyrthenope(card_mod=mod)
    for (k, v) in param:
        nope.card[k] += v
    data = nope.run()
    return data   

In [16]:
def runner(obs, params, cardMods=None, numThreads=cpu_count()):
    # This generates all of our runs, these will run automagically and will only halt when a corresponding .result() is called
    processes = np.empty((2, len(params), len(params), 4), dtype=object)
    with concurrent.futures.ThreadPoolExecutor(max_workers=numThreads) as exec:
        for i, (param1, step1) in enumerate(params.items()):
            processes[0, i, 0, 0] = exec.submit(_run, [(param1, -step1/2)], cardMods)
            processes[0, i, 0, 1] = exec.submit(_run, [(param1,  step1/2)], cardMods)
            for j, (param2, step2) in enumerate(params.items()):
                processes[1, i, j, 0] = exec.submit(_run, [(param1, -step1/2), (param2, -step2/2)], cardMods)
                processes[1, i, j, 1] = exec.submit(_run, [(param1,  step1/2), (param2, -step2/2)], cardMods)
                processes[1, i, j, 2] = exec.submit(_run, [(param1, -step1/2), (param2,  step2/2)], cardMods)
                processes[1, i, j, 3] = exec.submit(_run, [(param1,  step1/2), (param2,  step2/2)], cardMods)

    # derivative vectors
    d1Vec = pd.DataFrame()
    d2Vec = pd.DataFrame()

    #calculate the derivatives
    for i, (param1, step1) in enumerate(params.items()):
        v01 = processes[0, i, 0, 0].result()
        v02 = processes[0, i, 0, 1].result()
        
        d1 = pd.DataFrame((v02[obs] - v01[obs]) / step1).rename(index={0:param1})    
        d1Vec = pd.concat([d1Vec, d1])

        for j, (param2, step2) in enumerate(params.items()):
            v11 = processes[1, i, j, 0].result()
            v12 = processes[1, i, j, 1].result()
            v13 = processes[1, i, j, 2].result()
            v14 = processes[1, i, j, 3].result()

            d11 = (v12[obs] - v11[obs]) / step1
            d12 = (v14[obs] - v13[obs]) / step1
            d2 = pd.DataFrame((d12 - d11) / step2).rename(index={0:f'{param1},{param2}'})
            d2Vec = pd.concat([d2Vec, d2])

    errors = np.array([float(v) for v in obs.values()]).reshape(1, 2)
    ones = np.identity(len(obs))
    isigma2 = pd.DataFrame(
        np.linalg.inv(errors.T * ones * errors), 
        index=[list(obs.keys())], columns=[list(obs.keys())])
    
    # err = errors.T @ errors
    # # err[0,1] = err[1,0] = 0
    # isigma2 = pd.DataFrame(np.linalg.inv(err),
    #         index=[list(obs.keys())], columns=[list(obs.keys())]) 
    d2Ten = d2Vec.values.reshape(len(params), len(params), len(obs))
    
    # calculate fisher, dali3 and dali4
    # https://arxiv.org/pdf/1401.6892.pdf 15
    fisher = np.einsum('ia,ab,jb', d1Vec, isigma2, d1Vec)
    dali3  = np.einsum('ija,ab,kb', d2Ten, isigma2, d1Vec)
    dali4  = np.einsum('ija,ab,klb', d2Ten, isigma2, d2Ten)

    return fisher, dali3, dali4


In [17]:
def div_paper(obs, params): # https://arxiv.org/pdf/1505.01076.pdf
    Yp_eta10 = 0.24695 * 0.039 / 6.123
    Yp_tau   = 0.24695 * 0.729 / 879.4
    Yp_nnu   = 0.24695 * 0.163 / 3.0
        
    Yp_eta10_eta10 = 0.24695 * 0.039 / 6.123 * (0.039-1) / 6.123
    Yp_eta10_tau   = 0.24695 * 0.039 / 6.123 * 0.729 / 879.4
    Yp_eta10_nnu   = 0.24695 * 0.039 / 6.123 * 0.163 / 3.0
    
    Yp_tau_tau = 0.24695 * 0.729 / 879.4 * (0.729-1) / 879.4
    Yp_tau_nnu = 0.24695 * 0.729 / 879.4 * 0.163 / 3.0
    
    Yp_nnu_nnu = 0.24695 * 0.163 / 3.0 * (0.163-1) / 3.0
    
    
    
    DH_eta10 = 2.493*10**(-5)*(-1.634) / 6.123
    DH_tau   = 2.493*10**(-5)*(0.418) / 879.4
    DH_nnu   = 2.493*10**(-5)*(0.405) / 3.0
    
    DH_eta10_eta10 = 2.493*10**(-5)*(-1.634) / 6.123 * (-2.634) / 6.123
    DH_eta10_tau   = 2.493*10**(-5)*(-1.634) / 6.123 * 0.418 / 879.4
    DH_eta10_nnu   = 2.493*10**(-5)*(-1.634) / 6.123 * 0.405 / 3.0
    
    DH_tau_tau = 2.493*10**(-5)*(0.418) / 879.4 * (0.418-1) / 879.4
    DH_tau_nnu = 2.493*10**(-5)*(0.418) / 879.4 * 0.405 / 3.0
    
    DH_nnu_nnu = 2.493*10**(-5)*(0.405) / 3.0 * (0.405-1) / 3.0
    
    
    
    first = np.array([[DH_eta10, Yp_eta10],  
                      [DH_nnu, Yp_nnu], 
                      [DH_tau, Yp_tau]])
    second = np.array([[[DH_eta10_eta10, Yp_eta10_eta10], [DH_eta10_nnu, Yp_eta10_nnu], [DH_eta10_tau, Yp_eta10_tau]], 
                       [[DH_eta10_nnu,   Yp_eta10_nnu],   [DH_nnu_nnu, Yp_nnu_nnu],     [DH_tau_nnu, Yp_tau_nnu]],
                       [[DH_eta10_tau,   Yp_eta10_tau],   [DH_tau_nnu, Yp_tau_nnu],     [DH_tau_tau, Yp_tau_tau]]])
    
    errors = np.array([list(obs.values())])
    ones = np.identity(len(obs))
    isigma2 = pd.DataFrame(
        np.linalg.inv(errors.T * ones * errors), 
        index=[list(obs.keys())], columns=[list(obs.keys())])

    fisher = np.einsum('ia,ab,jb', first, isigma2, first)
    dali3 = np.einsum('ija,ab,kb', second, isigma2, first)
    dali4 = np.einsum('ija,ab,klb', second, isigma2, second)
    return fisher, dali3, dali4, first, second
    

In [18]:
def runner_nt(obs, params):
    d1Vec = pd.DataFrame()
    d2Vec = pd.DataFrame()
      
    for i, (param1, step1) in enumerate(params.items()):
        # fisher matrix
        vf1 = _run([(param1, -step1/2)])
        vf2 = _run([(param1,  step1/2)])
        d1 = pd.DataFrame((vf2[obs] - vf1[obs]) / step1).rename(index={0:param1})    
        d1Vec = pd.concat([d1Vec, d1])
        
        for j, (param2, step2) in enumerate(params.items()):
            vd1 = _run([(param1, -step1/2), (param2, -step2/2)])
            vd2 = _run([(param1,  step1/2), (param2, -step2/2)])
            vd3 = _run([(param1, -step1/2), (param2,  step2/2)])
            vd4 = _run([(param1,  step1/2), (param2,  step2/2)])
            
            d2 = pd.DataFrame(((vd2[obs] - vd1[obs]) / step1 - (vd4[obs] - vd3[obs]) / step1) / step2).rename(index={0:f'{param1},{param2}'})
            d2Vec = pd.concat([d2Vec, d2])

    errors = np.array([list(obs.values())])
    isigma2 = pd.DataFrame(
        np.linalg.inv(errors.T * np.identity(len(list(obs))) * errors), 
        index=[list(obs.keys())], columns=[list(obs.keys())])
    d2Ten = d2Vec.values.reshape(len(params), len(params), len(obs))
    print(d2Vec)
    print(d2Ten)
    
    # calculate fisher, dali3 and dali4
    # https://arxiv.org/pdf/1401.6892.pdf 15
    fisher = np.einsum('ia,ab,jb', d1Vec, isigma2, d1Vec)
    dali3 = np.einsum('ija,ab,kb', d2Ten, isigma2, d1Vec)
    dali4 = np.einsum('ija,ab,klb', d2Ten, isigma2, d2Ten)

    return fisher, dali3, dali4



In [19]:
pd.set_option("display.precision", 12)
print(_run([]))

# fisher, dali3, dali4 = runner(observables, parameters)

   N_nu  xie  xix    tau  rholmbd  eta10     OmegaBh^2           phie  \
0   3.0  0.0  0.0  879.4      0.0  6.123  0.0223662699  39.4127139257   

               thetah            tnuh                 nbh             N/H  \
0  8.685496385130e-26  0.714957501589  5.083340801740e-16  0.000000000924   

              Y_H            H2/H            H3/H           He3/H  \
0  0.754479003909  0.000025218605  0.000000081054  0.000010338307   

              Y_p               Li6/H           Li7/H           Be7/H  
0  0.246830312639  1.095289397790e-14  0.000000000466  0.000000000438  


In [20]:
fisher2, d32, d42, f, s = div_paper(observables, parameters)
print(f'fisher: \n{fisher2}')
print(f'dali3: \n{d32}')
print(f'dali4: \n{d42}')

fisher: 
[[ 4.91942298e+02 -2.47465605e+02 -8.55825527e-01]
 [-2.47465605e+02  1.37106770e+02  6.14798754e-01]
 [-8.55825527e-01  6.14798754e-01  4.17947474e-03]]
dali3: 
[[[-2.11582119e+02  1.06815481e+02  3.73658965e-01]
  [ 6.63997367e+01 -3.35142609e+01 -1.17159876e-01]
  [ 2.33886708e-01 -1.17159876e-01 -3.99677215e-04]]

 [[ 6.63997367e+01 -3.35142609e+01 -1.17159876e-01]
  [ 4.89742742e+01 -2.81005062e+01 -1.35783487e-01]
  [-1.17159876e-01  6.91494316e-02  3.52941338e-04]]

 [[ 2.33886708e-01 -1.17159876e-01 -3.99677215e-04]
  [-1.17159876e-01  6.91494316e-02  3.52941338e-04]
  [ 5.73515332e-04 -3.46170277e-04 -1.83973327e-06]]]
dali4: 
[[[[ 9.10120364e+01 -2.85616284e+01 -1.00578660e-01]
   [-2.85616284e+01 -2.11683704e+01  5.06987563e-02]
   [-1.00578660e-01  5.06987563e-02 -2.48410110e-04]]

  [[-2.85616284e+01  8.96328672e+00  3.15643653e-02]
   [ 8.96328672e+00  6.64121379e+00 -1.59047897e-02]
   [ 3.15643653e-02 -1.59047897e-02  7.79248537e-05]]

  [[-1.00578660e-01  3.15

Saver

In [21]:
if save:
    saveData = {
        'cosmoFid': {'eta10': 6.123, 'N_nu': 3.0, 'tau': 879.4}, #{'eta10': 6.13832, 'N_nu': 3.0, 'tau': 879.4}, 
        'fisherGaussian': {lo_experment: {lo_spec_id: fisher}},
        'DALI3Gaussian':  {lo_experment: {lo_spec_id: dali3}},
        'DALI4Gaussian': {lo_experment: {lo_spec_id: dali4}},
        'nfisherGaussian': {lo_experment: {lo_spec_id: fisher2}},
        'nDALI3Gaussian': {lo_experment: {lo_spec_id: d32}},
        'nDALI4Gaussian': {lo_experment: {lo_spec_id: d42}} }

    with open(save_file, 'wb') as file:
        pickle.dump(saveData, file)

In [22]:
print(f'first div: \n{f}')
print(f'second div: \n{s}')
print(f'old fisher: \n{fisher}')
print(f'new fisher: \n{fisher2}')
print(f'old dali3: \n{dali3}')
print(f'new dali3: \n{d32}')
print(f'old dali4: \n{dali4}')
print(f'new dali4: \n{d42}')
print(f'(fisher2 - fisher)/fisher: \n{(fisher2 - fisher)/fisher}')
print(f'(d32 - dali3)/dali3: \n{(d32 - dali3)/dali3}')
print(f'(d42 - dali4)/dali4: \n{(d42 - dali4)/dali4}')

first div: 
[[-6.65288584e-06  1.57292994e-03]
 [ 3.36555000e-06  1.34176167e-02]
 [ 1.18498294e-08  2.04715204e-04]]
second div: 
[[[ 2.86194697e-06 -2.46870108e-04]
  [-8.98139588e-07  8.54625265e-05]
  [-3.16227687e-09  1.30391849e-06]]

 [[-8.98139588e-07  8.54625265e-05]
  [-6.67500750e-07 -3.74351505e-03]
  [ 1.59972697e-09  1.11228594e-05]]

 [[-3.16227687e-09  1.30391849e-06]
  [ 1.59972697e-09  1.11228594e-05]
  [-7.84239337e-12 -6.30859906e-08]]]
old fisher: 
[[ 5.06049552e+02 -2.55554979e+02 -8.78798042e-01]
 [-2.55554979e+02  1.41617640e+02  6.27457347e-01]
 [-8.78798042e-01  6.27457347e-01  4.21131257e-03]]
new fisher: 
[[ 4.91942298e+02 -2.47465605e+02 -8.55825527e-01]
 [-2.47465605e+02  1.37106770e+02  6.14798754e-01]
 [-8.55825527e-01  6.14798754e-01  4.17947474e-03]]
old dali3: 
[[[ 5.66979301e+02 -2.63640317e+02 -6.52961142e-01]
  [-3.50799589e+02  2.18858592e+02  1.21892504e+00]
  [-3.51751485e+00  2.03871595e+00  9.94437610e-03]]

 [[-3.50799589e+02  2.18858592e+02 